<a href="https://colab.research.google.com/github/feliciacia/Data-Science/blob/main/LGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import lightgbm as lgb


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
train=pd.read_csv('gdrive/My Drive/data_sciencehw1/train.csv')
test=pd.read_csv('gdrive/My Drive/data_sciencehw1/test.csv')
train['Attribute8'].unique()
train.head()

,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Attribute11,Attribute12,Attribute13,Attribute14,Attribute15,Attribute16,Attribute17,Attribute18,Attribute19,Attribute20,Attribute21,Attribute22,Attribute23
0,2008-12-01,2,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-17,2,14.1,20.9,0.0,NaN,NaN,ENE,22.0,SSW,E,11.0,9.0,69.0,82.0,1012.2,1010.4,8.0,1.0,17.2,18.1,No,Yes
2,2008-12-20,2,9.8,25.6,0.0,NaN,NaN,SSE,26.0,SE,NNW,17.0,6.0,45.0,26.0,1019.2,1017.1,NaN,NaN,15.8,23.2,No,No
3,2008-12-30,2,12.5,24.2,1.2,NaN,NaN,WNW,50.0,WSW,SW,11.0,22.0,78.0,70.0,1005.6,1003.4,8.0,8.0,12.5,18.2,Yes,No
4,2009-01-22,2,24.4,34.0,0.6,NaN,NaN,NW,98.0,N,NNW,26.0,48.0,60.0,39.0,1005.3,997.8,4.0,1.0,26.1,33.1,No,Yes


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
x=train.drop(['Attribute23', 'Attribute1','Attribute2','Attribute3','Attribute4','Attribute6','Attribute8','Attribute10','Attribute11','Attribute12', 'Attribute13','Attribute14','Attribute16','Attribute18','Attribute19','Attribute20','Attribute21','Attribute22'],axis=1) #1=col
test_data=test.drop(['Attribute1','Attribute2','Attribute3','Attribute4','Attribute6','Attribute8','Attribute10','Attribute11','Attribute12', 'Attribute13','Attribute14','Attribute16','Attribute18','Attribute19','Attribute20','Attribute21','Attribute22'],axis=1) #1=col
y=train.Attribute23

x.head(10)

,Attribute5,Attribute7,Attribute9,Attribute15,Attribute17
0,0.6,NaN,44.0,22.0,1007.1
1,0.0,NaN,22.0,82.0,1010.4
2,0.0,NaN,26.0,26.0,1017.1
3,1.2,NaN,50.0,70.0,1003.4
4,0.6,NaN,98.0,39.0,997.8
5,0.0,NaN,24.0,10.0,1014.6
6,0.0,NaN,24.0,13.0,1011.8
7,0.2,NaN,56.0,21.0,1007.4
8,0.4,NaN,37.0,31.0,1007.8
9,0.0,NaN,41.0,30.0,1008.5


In [ ]:
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.3)
x_train.head()

,Attribute5,Attribute7,Attribute9,Attribute15,Attribute17
4713,0.0,0.2,24.0,88.0,1014.3
5457,0.0,11.4,41.0,99.0,1011.6
12132,0.4,8.7,46.0,30.0,1010.2
6962,0.0,NaN,50.0,23.0,1014.4
9495,0.0,0.6,44.0,60.0,1018.7


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param1 = {'max_depth':[5,10,20,30,40], 
          'num_leaves':[10,20,30,40,50,60]}
grid1 = GridSearchCV(estimator = lgb.LGBMClassifier(learning_rate=0.01, scale_pos_weight=4.38),param_grid=param1,scoring='roc_auc')
grid1.fit(x,y)
grid1.best_params_, grid1.best_score_

({'max_depth': 20, 'num_leaves': 40}, 0.8528113559418962)

In [ ]:
param2 = {'n_estimators':[200,225,250,275,300,325,350,375,400], 
          'min_child_weight':[10,20,30,40,50]}
grid2 = GridSearchCV(estimator = lgb.LGBMClassifier(learning_rate=0.01, scale_pos_weight=4.38, max_depth=20,num_leaves=40),param_grid=param2,scoring='roc_auc')
grid2.fit(x,y)
grid2.best_params_, grid2.best_score_

({'min_child_weight': 30, 'n_estimators': 300}, 0.8556671184076732)

In [ ]:
param3 = {'reg_alpha':[0,1e-5,1e-3,1e-1,1,10],
          'reg_lambda':[0,1e-5,1e-3,1e-1,1,10,100]}
grid3 = GridSearchCV(estimator = lgb.LGBMClassifier(learning_rate=0.01, scale_pos_weight=4.38, max_depth=20,num_leaves=40,n_estimators=300,min_child_weight=30),param_grid=param3,scoring='roc_auc')
grid3.fit(x,y)
grid3.best_params_, grid3.best_score_

({'reg_alpha': 0.1, 'reg_lambda': 0.1}, 0.8557139701038399)

In [ ]:
param4 = {'boosting_type':['gbdt','dart','goss']}
grid4 = GridSearchCV(estimator = lgb.LGBMClassifier(learning_rate=0.01, scale_pos_weight=4.38, max_depth=20,num_leaves=40,n_estimators=300,min_child_weight=30,reg_alpha=0.1,reg_lambda=0.1),param_grid=param4,scoring='roc_auc')
grid4.fit(x,y)
grid4.best_params_, grid4.best_score_

({'boosting_type': 'goss'}, 0.8559979221482836)

In [ ]:
dc= lgb.LGBMClassifier(boosting_type='goss',learning_rate = 0.01, scale_pos_weight=4.38, max_depth=20, num_leaves=40, n_estimators=300,min_child_weight=30, reg_lambda=0.1,reg_alpha=0.1)

dc=dc.fit(x,y)
metrics.accuracy_score(dc.predict(x_test),y_test) 

0.8057722308892356

In [ ]:
y_pred = dc.predict(test_data)

In [ ]:
for data in range(len(y_pred)):
  if(y_pred[data]=='No'):
    y_pred[data] = 0
  else:
    y_pred[data] = 1

In [ ]:

y_pred_Data = pd.DataFrame(y_pred)
y_pred_Data = y_pred_Data.reset_index()
y_pred_Data['index'].astype('str')
y_pred_Data.to_csv('gdrive/My Drive/data_sciencehw1/result1.csv',index=False)
y_pred_Data.head(5)

,index,0
0,0,0
1,1,1
2,2,0
3,3,0
4,4,0


In [ ]:
y_pred_Data = pd.DataFrame(y_pred, columns = ['Ans'])
y_pred_Data
submit = pd.read_csv('gdrive/My Drive/data_sciencehw1/ex_submit.csv')
submit = submit.drop('ans', axis = 1)
submit.head()
submitt = pd.concat([submit, y_pred_Data], axis = 1)
submitt

,id,Ans
0,0.0,0
1,1.0,1
2,2.0,0
3,3.0,0
4,4.0,0
...,...,...
785,785.0,0
786,786.0,0
787,787.0,1
788,788.0,1


In [ ]:
submitt.to_csv('gdrive/My Drive/data_sciencehw1/submitt.csv', index = False)

In [ ]:
print(dc.feature_importances_)
x_train.head()


[1597 2341 1978 2568 3216]


,Attribute5,Attribute7,Attribute9,Attribute15,Attribute17
4713,0.0,0.2,24.0,88.0,1014.3
5457,0.0,11.4,41.0,99.0,1011.6
12132,0.4,8.7,46.0,30.0,1010.2
6962,0.0,NaN,50.0,23.0,1014.4
9495,0.0,0.6,44.0,60.0,1018.7
